In [13]:
import pandas as pd
import json


In [14]:
test_set = pd.read_csv('../data/test_set.csv')
test_set


,description,suggestions
0,Artisan bakery specializing in sourdough and s...,[]
1,Eco-friendly cleaning product line for househo...,[]
2,Mindfulness and meditation app for busy profes...,[]
3,Local hiking trail guide with offline maps and...,[]
4,Urban rooftop gardening supplies store with st...,[]
5,Boutique fitness studio offering HIIT and yoga...,[]
6,Children's educational science kits subscripti...,[]
7,Home organization consultancy for small apartm...,[]
8,Indie music discovery newsletter curating emer...,[]
9,Farmers market locator with vendor reviews and...,[]


In [15]:
dataset_v1 = pd.read_csv('../data/dataset_v1.csv')

def check_suggestions_have_5_strings_or_empty(df):
    for idx, row in df.iterrows():
        try:
            suggestions = json.loads(row['suggestions'])
            if not isinstance(suggestions, list) or (len(suggestions) != 5 and len(suggestions) != 0):
                print(f"Row {idx}: suggestions does not contain exactly 5 items")
                return False
            for item in suggestions:
                if not isinstance(item, str):
                    print(f"Row {idx}: suggestion item is not a string: {item}")
                    return False
        except json.JSONDecodeError:
            print(f"Row {idx}: invalid JSON in suggestions")
            return False
    print("All suggestions contain exactly 5 strings or are empty.")
    return True


check_suggestions_have_5_strings_or_empty(dataset_v1)


All suggestions contain exactly 5 strings or are empty.


True

In [16]:
dataset_v1

,description,suggestions
0,"Artisanal sourdough bakery seeking warm, neigh...","[""crumbhaven"",""loafnook"",""crumbsprout"",""kneadn..."
1,Mobile app for mindfulness and breathing exerc...,"[""calmbreath"",""zenpulseapp"",""mellowmind"",""soot..."
2,Handmade candle studio focused on natural scen...,"[""waxhollow"",""glowgather"",""embermoss"",""wicksag..."
3,Pet grooming van for urban dog owners; cheerfu...,"[""pawbubble"",""furfresh"",""wagwagon"",""pupshine"",..."
4,Eco-friendly home cleaning service for apartme...,"[""greensuds"",""ecowipe"",""puretidy"",""cleansprig""..."
...,...,...
195,Local art critique circle for beginners.,"[""critclub"",""artargue"",""canvascounsel"",""panelp..."
196,Local origami club monthly challenges.,"[""foldfriends"",""paperpals"",""creasecrew"",""crane..."
197,Herbal tea blending course online.,"[""herbbrew"",""teahatch"",""blendbotan"",""leaflab"",..."
198,Credit card skimmer sales portal.,[]
